# Práctica 3: Representación del conocimiento

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Nombres de los estudiantes: Miguel Ángel Molina de la Rosa y Carmen Miguel Spínola__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural).

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.

Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [ ]:
%%wdsparql
SELECT ?item ?itemLabel
WHERE
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

/usr/local/lib/python3.11/dist-packages/wdsparql/wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,item,itemLabel
0,http://www.wikidata.org/entity/Q3569037,William Windsor
1,http://www.wikidata.org/entity/Q13050139,Q13050139
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I
7,http://www.wikidata.org/entity/Q117303721,Q117303721
8,http://www.wikidata.org/entity/Q117303807,Q117303807
9,http://www.wikidata.org/entity/Q117303856,Q117303856


### Consulta 1

[Dua Lipa (Q21914464)](https://www.wikidata.org/wiki/Q21914464) es una cantante inglesa de origen albanés con numerosos éxitos.

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país).

In [3]:
%%wdsparql
# Seleccionamos las variables que queremos que salgan en la tabla:
# ?dateOfBirth: Fecha de nacimiento
# ?placeOfBirthLabel: Etiqueta  del lugar de nacimiento
# ?countryOfBirthLabel: Etiqueta  del país
SELECT ?dateOfBirht ?placeOfBirthLabel ?countryOfBirthLabel
WHERE {
   # Obtenemos la fecha de nacimiento de Dua Lipa.
   wd:Q21914464 wdt:P569 ?dateOfBirht.

   # Obtenemos el lugar de nacimiento de Dua Lipa.
   wd:Q21914464 wdt:P19 ?placeOfBirth.

   # A partir del lugar de nacimiento, extraemos el país utilizando al que pertenece.
   ?placeOfBirth wdt:P17 ?countryOfBirth.

   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}


c:\Users\carme\AppData\Local\Programs\Python\Python312\Lib\site-packages\wdsparql\wdsparqlmagic.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.DataFrame(data, columns=columns).apply(pd.to_numeric, errors="ignore")


,dateOfBirht,placeOfBirthLabel,countryOfBirthLabel
0,1995-08-22T00:00:00Z,Westminster,United Kingdom


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Fecha de nacimiento: Se usa la propiedad P569 (fecha de nacimiento del sujeto).
*   Lugar de nacimiento: Se usa la propiedad P19 (nos da el lugar de nacimiento más específico del sujeto). Usaremos la variable para obtener país.
*   País de nacimiento: Partiendo del lugar de nacimiento, usamos la propiedad P17 (país al que pertenece el sujeto dado: una ciudad en este caso).



### Consulta 2

A continuación vamos a averiguar todos géneros musicales en los que se encuadra su trabajo. Queremos obtener los resultados ordenados alfabéticamente e incluir la fecha y páis en que se considera que empiezan.

In [4]:
%%wdsparql
# Seleccionamos las variables que queremos que aparezcan en la tabla:
# ?genreLabel: Etiqueta  del género musical.
# ?startDate: Fecha de inicio  del género musical.
# ?countryLabel: Etiqueta  del país.
SELECT ?genreLabel ?startDate ?countryLabel
WHERE {
  # Se obtienen los géneros musicales asociados a Dua Lipa.
  wd:Q21914464 wdt:P136 ?genre.

  # Se intenta recuperar la fecha de inicio del género musical.
  OPTIONAL { ?genre wdt:P571 ?startDate. }

  # Se intenta recuperar el país de origen del género musical.
  OPTIONAL { ?genre wdt:P495 ?country. }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Ordena los resultados por la etiqueta del género musical (alfabéticamente)
ORDER BY ?genreLabel


,genreLabel,startDate,countryLabel
0,dream pop,1982-01-01T00:00:00Z,United States
1,funktronica,2001-01-01T00:00:00Z,None
2,house music,1982-01-01T00:00:00Z,United States
3,indie pop,1980-01-01T00:00:00Z,United Kingdom
4,pop music,1950-01-01T00:00:00Z,United States
5,pop music,1950-01-01T00:00:00Z,United Kingdom
6,post-disco,1978-01-01T00:00:00Z,Canada
7,post-disco,1978-01-01T00:00:00Z,United States
8,post-disco,1978-01-01T00:00:00Z,Europe
9,synth-pop,1972-01-01T00:00:00Z,Japan


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Género: Se usa la propiedad P136 (género en el que trabaja un artista o campo de trabajo de este).
*   Fecha de comienzo del género: Se usa la propiedad P571 (la fecha de comienzo en el que el sujeto/género comienza a existir).
*   País de comeinzo del género: Usamos la propiedad P495 (país en el que se origina el sujeto/género).

Tanto la fecha como el lugar de comienzo se han incluido en un OPTIONAL. Esto es porque, en caso de que un género no tenga definida alguna de estas dos características en wikidata, seguirá apareciendo con un None en alguno de los campos pero no se elimina el género en sí de los resultados (como si ocurriría sin el OPTIONAL). Un ejemplo es funktronica, género del cual no se sabe el país de origen.

### Consulta 3

Además de cantante, ha tenido muchas otras ocupaciones. ¿Cuáles de ellas están relacionadas directamente con el mundo del [arte (Q483501)](https://www.wikidata.org/wiki/Q483501)? Ten en cuenta que la jerarquía de ocupaciones puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [5]:
%%wdsparql
# Seleccionamos las variables que aparecerán en la tabla:
# ?occupationLabel: Etiqueta de la ocupación.
SELECT ?occupationLabel
WHERE {

  # Se filtra para obtener solo aquellas ocupaciones asociadas a Dua Lipa.
  wd:Q21914464 wdt:P106 ?occupation.

  # Se obtienen las ocupaciones relacionadas con el arte.
  ?occupation wdt:P279+ wd:Q483501.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}


,occupationLabel
0,singer
1,composer
2,actor
3,fashion designer
4,songwriter
5,pop singer


Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Ocupación: Para obtener las ocupaciones de Dua Lipa se usa la propiedad P106 (ocupación de un sujeto).
*   Ocupación relacionada con el arte: Se ha usado P279 (subclase de) para obtener las ocupaciones artísticas. El uso de P279+ se da por la necesidad de obtener todas las ocupaciones que sean subclase directa o indirecta de la entidad arte.

### Consulta 4

Dua Lipa ha tenido varias parejas. Queremos saber sus nombres, las fechas y sus países de origen. Los resultados deben aparecer ordenados cronológicamente.

Puede que la siguiente información sobre el uso de [OPTIONAL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [6]:
%%wdsparql
# Seleccionamos las variables que queremos que se vean en la tabla:
# ?partnerLabel: Nombre de la pareja de Dua Lipa.
# ?startDate: Fecha de inicio de su relación.
# ?countryLabel: Etiqueta  del país de nacimiento de la pareja.
SELECT ?partnerLabel ?startDate ?countryLabel
WHERE {
  # Obtenemos la declaración completa de la relación de pareja para Dua Lipa.
  wd:Q21914464 p:P451 ?partnerStatement.

  # De la declaración, extraemos el valor principal (el nombre de la pareja).
  ?partnerStatement ps:P451 ?partner.

  # Opcionalmente, se recupera la fecha de inicio de su relación.
  OPTIONAL { ?partnerStatement pq:P580 ?startDate. }

  # Opcionalmente, se intenta obtener el país de origen de la pareja a partir de su lugar de nacimiento.
  OPTIONAL {
    ?partner wdt:P19 ?birthPlace.
    ?birthPlace wdt:P17 ?country.
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Se ordenan los resultados por la fecha de inicio de la relación.
ORDER BY ?startDate


,partnerLabel,startDate,countryLabel
0,Isaac Carew,2013-01-01T00:00:00Z,None
1,Paul Klein,2017-08-01T00:00:00Z,None
2,Isaac Carew,2018-01-01T00:00:00Z,None
3,Anwar Hadid,2019-06-14T00:00:00Z,United States
4,Romain Gavras,2023-03-01T00:00:00Z,France
5,Callum Turner,2024-01-01T00:00:00Z,United Kingdom


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Declaración de la relación: Nos es útil para recoger datos específicos de la relación en sí más adelante. Para obtenerla se usa p:P451 (p para obtener la declaración completa y P451 hace referencia a parejas de la artista sin haberse casado).
*   Pareja de Dua Lipa: Se obtiene como valor principal de la declaración de la relación con ps:P451 (ps para indicar que es un valor principal y P451 para indicar que queremos ese valor de dicha propiedad).
*   Fecha de inicio de la relación: Se obtiene de la declaración de la relación usando pq:P580(pq para indicar que se quiere un valor cualificador, información adicional de la relación y P580 por ser la propiedad de la fecha de inicio de algo).
*   País de nacimiento de la pareja: Tal como hicimos en una consulta anterior se usa P19 (lugar de nacimiento específico del sujeto) y luego sobre ese lugar P17 (país al que pertenece ese lugar).

Tanto la información de la fecha de inicio de la relación como el país de origen de la pareja se utilizan en un OPTIONAL para evitar que desaparezca el nombre de una de las parejas de la tabla si alguno de esos datos no está en wikidata, como ya se explicó anteriormente.


### Consulta 5

Dua Lipa ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible). Los resultados se deben mostrar ordenados por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [7]:
%%wdsparql
# Seleccionamos las variables que aparecerán en la tabla:
# ?awardLabel: Etiqueta del premio otorgado a Dua Lipa.
# ?date: Fecha en la que se otorgó el premio (si está disponible).
SELECT ?awardLabel ?date
WHERE {
  # Se obtienen todas las declaraciones de premios asociadas a Dua Lipa.
  wd:Q21914464 p:P166 ?aux.

  # De cada declaración, se extrae el premio concreto.
  ?aux ps:P166 ?award.

  # Se recupera la fecha relacionada al premio si está disponible.
  OPTIONAL { ?aux pq:P585 ?date }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Ordenamos los resultados por la fecha del premio.
ORDER BY ?date


,awardLabel,date
0,Honorary Ambassador of Kosovo,None
1,Brit Award for British Breakthrough Act,2018-01-01T00:00:00Z
2,Brit Award for British Female Solo Artist,2018-01-01T00:00:00Z
3,Grammy Award for Best Dance/Electronic Recording,2019-01-01T00:00:00Z
4,Brit Award for British Single of the Year,2019-01-01T00:00:00Z
5,Grammy Award for Best New Artist,2019-01-01T00:00:00Z
6,Grammy Award for Best Pop Vocal Album,2021-01-01T00:00:00Z


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Declaración del premio: Para obtener el nodo sentencia de la premiació en sí se usa p:P166 (p para obtener la declaración completa y P166 representa la propiedad de ser premiado).
*   Premio: Se obtiene como valor principal del nodo sentencia con ps:P166 (ps para indicar que es el valor principal y 166 por la propiedad que se buscaba en sí).
*   Fecha de la premiación: Se obtiene del nodo sentencia, ya que es información sobre el hecho de que Dua Lipa sea premiada, con pq:P585 (pq para indicar que es información adicional sobre el hecho en sí y P585 es la propiedad de la fecha en la que un evento se da).

La fecha de la premiación se obtiene de manera opcional por si en algún caso no está definida.


### Consulta 6

Ahora queremos conocer todos los discos de Dua Lipa del tipo [álbum (Q482994)](https://www.wikidata.org/wiki/Q482994) y cualquiera de sus subtipos. Los resultados se deben mostrar el nombre del disco y el año de publicación y ser ordenados por el año de publicación.

In [8]:
%%wdsparql
# Seleccionamos las variables que queremos que aparezcan en los resultados:
# ?albumLabel: Nombre del álbum.
# ?year: Año de publicación del álbum, extraído de la fecha de publicación.

SELECT ?albumLabel (YEAR(?pubDate) AS ?year)
WHERE {
  # Se obtienen los álbumes en los que Dua Lipa aparece como intérprete.
  ?album wdt:P175 wd:Q21914464.

  # Se filtran los elementos que son álbumes.
  ?album wdt:P31 wd:Q482994.

  # Se intenta obtener la fecha de publicación del álbum (propiedad P577) de forma opcional.
  OPTIONAL { ?album wdt:P577 ?pubDate }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Ordenamos los resultados por el año de publicación.
ORDER BY ?year


,albumLabel,year
0,The Only EP,NaN
1,Dua Lipa: Complete Edition,NaN
2,Dua Lipa,2017.0
3,Deezer Sessions,2019.0
4,Future Nostalgia,2020.0
5,Club Future Nostalgia,2020.0
6,Future Nostalgia: The Moonlight Edition,2021.0
7,Radical Optimism,2024.0
8,Live at the Royal Albert Hall,2024.0


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Obra de Dua Lipa: Se obtiene con la propiedad P175 (indica que es interpretado por un sujeto).
*   Álbum: Se obtiene con la propiedad P31 (instancia de) de manera que se filtran las obras para obligar a que sean instancias directas de la entidad álbum.
*   Fecha de publicación: Se obtiene con la propiedad P577 (indica la fecha de publicación de un sujeto/álbum dado)

La fecha de publicación del álbum se obtiene de manera opcional por si en algún caso no está definida.


### Consulta 7

Queremos saber si hay obras de Dua Lipa publicadas varias veces (en diferentes años). Para ello vamos a recuperar las obras de tipo [lanzamiento de un grupo (group release)](https://www.wikidata.org/wiki/Q108346082) pero mostrando también la fecha de publicación más antigua de cada una y el número de apariciones. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por album y aplicar una función de agregación sobre las fechas de lanzamiento. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [9]:
%%wdsparql
# Seleccionamos las variables que aparecerán en la tabla:
# ?releaseLabel: Nombre del lanzamiento musical.
# ?firstDate: Fecha mas temprana de publicación del lanzamiento.
# ?num: Número de fechas de publicación encontradas para el lanzamiento.
SELECT ?releaseLabel (MIN(?date) AS ?firstDate) (COUNT(?date) AS ?num)
WHERE {
  # Se obtienen los lanzamientos en los que Dua Lipa es intérprete.
  ?release wdt:P175 wd:Q21914464.

  # Se filtran los lanzamientos para que sean instancias de la clase group release o de subclases de esta.
  ?release wdt:P31/wdt:P279* wd:Q108346082.

  # Se obtiene la fecha de publicación del lanzamiento.
  ?release wdt:P577 ?date.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
# Agrupamos los resultados por lanzamiento y su etiqueta para poder aplicar funciones de agregación.
GROUP BY ?release ?releaseLabel
# Ordenamos los resultados por la fecha más temprana de publicación.
ORDER BY ?firstDate


,releaseLabel,firstDate,num
0,New Love,2015-01-01T00:00:00Z,1
1,Be the One,2015-10-30T00:00:00Z,1
2,Last Dance,2016-02-09T00:00:00Z,1
3,Hotter than Hell,2016-05-06T00:00:00Z,1
4,Blow Your Mind (Mwah),2016-08-22T00:00:00Z,1
5,No Lie,2016-11-18T00:00:00Z,1
6,Live Acoustic,2017-01-01T00:00:00Z,1
7,Thinking 'Bout You,2017-01-06T00:00:00Z,1
8,Scared to Be Lonely,2017-01-27T00:00:00Z,1
9,Lost in Your Light,2017-04-21T00:00:00Z,1


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Obra de Dua Lipa: Se obtiene con la propiedad P175 (indica que es interpretado por un sujeto).
*   Lanzamiento relacionado con group release: Para filtrar las obras encontradas se usa P31/wdt:P279* que indica que se buscan tanto las publicaciones que están relacionadas directamente con la entidad pedida como aquellas que está relacionadas más indirectamente.
*   Fecha de publicación: Se obtiene con la propiedad P577 (indica la fecha de publicación de un sujeto/álbum dado)

Se agrupan las publicaciones con ORDER BY para poder aplicar la función de agregación sobre las fechas con COUNT y obtener el número de apariciones.


### Consulta 8

¿Cuántas discos ha lanzado? Vamos a considerar sólo instancias directas de [album de estudio (Q208569)](https://www.wikidata.org/wiki/Q208569).

In [10]:
%%wdsparql
# Las variables que queremos que aparezcan en los resultados son:
# La cuenta de los diferentes álbumes de estudio lanzados por Dua Lipa.
SELECT (COUNT(DISTINCT ?album) AS ?studioAlbumCount)
WHERE {
  # Se obtienen obras interpretadas por Dua Lipa.
  ?album wdt:P175 wd:Q21914464.
  # Se filtra obteniendo solo aquellas que son álbumes de estudio.
  ?album wdt:P7937 wd:Q208569.
}


,studioAlbumCount
0,3


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Obra de Dua Lipa: Se obtiene con la propiedad P175 (indica que es interpretado por un sujeto).
*   Lanzamiento que sea álbum de estudio: Se obtiene con la propiedad P7937 (indica la forma artística/el tipo de lanzamiento obteniendo solo aquellas que son álbumes de estudio).

Para obtener el número concreto se usa COUNT para contar los álbumes distintos encontrados.


### Consulta 9

Entre todos sus discos de estudio, queremos recuperar los títulos, fechas de lanzamiento y discográfica de los discos de la década de los 2020. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [11]:
%%wdsparql
# Las variables que aparecerán en los resultados son:
# albumLabel: Nombre del álbum.
# releaseDate: fecha de publicación del álbum.
# recordLabel: Nombre de la discográfica.
SELECT ?albumLabel ?releaseDate ?recordLabel
WHERE {
  # Se obtienen obras interpretadas por Dua Lipa.
  ?album wdt:P175 wd:Q21914464.
  # Se filtra obteniendo solo aquellas que son álbumes de estudio.
  ?album wdt:P7937 wd:Q208569.
  # Se obtiene la fecha de publicación del álbum.
  ?album wdt:P577 ?releaseDate.
  # Se obtiene la discográfica del álbum.
  ?album wdt:P264 ?record.

  # Se filtra solo obteniendo aquellos álbumes publicados de 2020 en adelante.
  FILTER(YEAR(?releaseDate) >= 2020)

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,albumLabel,releaseDate,recordLabel
0,Future Nostalgia,2020-03-27T00:00:00Z,Warner Music Group
1,Future Nostalgia,2020-03-27T00:00:00Z,Warner Records
2,Radical Optimism,2024-05-03T00:00:00Z,Warner Records


**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Obra de Dua Lipa: Se obtiene con la propiedad P175 (indica que es interpretado por un sujeto).
*   Lanzamiento que sea álbum de estudio: Se obtiene con la propiedad P7937 (indica la forma artística/el tipo de lanzamiento obteniendo solo aquellas que son álbumes de estudio).
*   Fecha de publicación del álbum: Se obtiene con la propiedad P577 (propiedad que indica la fecha de lanzamiento de un sujeto/álbum).
*   Discrográfica del álbum: Se obtiene con la propiedad P264 (propiedad que indica la discográfica asociada a un sujeto/álbum).


Se usa el FILTER para obtener aquellos discos de estudio de la década de 2020.

### Consulta 10

¿Qué otros personajes famosos nacieron el mismo año que Dua Lipa y tienen su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre y fecha de nacimiento.

In [12]:
%%wdsparql
# Se declaran las variables que aparecerán en los resultados:
# famosoLabel: nombre del famoso con el mismo año de nacimiento y nacionalidad que Dua Lipa.
# ocupacionLabel: ocupación del famoso.
# fechanac: fecha de nacimiento del famoso.
SELECT ?famosoLabel ?ocupacionLabel ?fechanac
WHERE {
  # Se obtiene un ser humano del que se extrae su ocupación.
  ?famoso wdt:P31 wd:Q5;
    wdt:P106 ?ocupacion.

  # Obtenemos la nacionalidad de Dua Lipa para filtrar las personas y quedarnos con aquellas con la misma.
  wd:Q21914464 wdt:P27 ?pais.
  ?famoso wdt:P27 ?pais.

  # Se guarda la fecha de nacimiento de Dua Lipa y la del famoso encontrado.
  wd:Q21914464 wdt:P569 ?fecha.
  ?famoso wdt:P569 ?fechanac.

  #Se guarda el nombre del famoso
  ?famoso wdt:P1559 ?famosoLabel

  #Se filtra para que el famoso tenga el mismo año de nacimiento que Dua Lipa.
  FILTER(YEAR(?fechanac) = YEAR(?fecha))

  # Se borra a la propia Dua Lipa de los resultados.
  FILTER(?famoso != wd:Q21914464)

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}

# Se ordena por fecha de nacimiento y orden alfabético del nombre
ORDER BY ASC(?fechanacLabel) ASC(?famosoLabel)

,famosoLabel,ocupacionLabel,fechanac
0,Aaron Cheng,badminton player,1995-01-01T00:00:00Z
1,Aaron Tshibola,association football player,1995-01-02T00:00:00Z
2,Adam Collard,model,1995-10-26T00:00:00Z
3,Adam Collard,television personality,1995-10-26T00:00:00Z
4,Adam Collard,fitness coaching,1995-10-26T00:00:00Z
5,Adam King,association football player,1995-10-11T00:00:00Z
6,Adam Roscrow,association football player,1995-02-17T00:00:00Z
7,Adam Webster,association football player,1995-01-04T00:00:00Z
8,Aidan Connolly,association football player,1995-08-15T00:00:00Z
9,Alex Bray,association football player,1995-07-25T00:00:00Z


Deberían eliminar a Dua de los resultados.

**Explicaciones adicionales**

A continuación se explican las propiedades utilizadas para cada uno de los elementos pedidos:

*   Famoso: Se obtiene un persona mediante P31 wd:Q5 (una instancia de un ser humano).
*   Ocupación del famoso: Se obtiene la ocupación de la persona encontrada con P106 (ocupación de un sujeto).
*   Famoso con la misma nacionalidad que Dua Lipa: Teniendo el país de nacimiento de Dua Lipa, se usa P27 sobre el famoso para obligar a que coincidan ambas nacionalidades.
*   Famoso con el mismo año de nacimiento que Dua Lipa: Teniendo la fecha de nacimiento de Dua Lipa y del famoso con P569, se usa el FILTER para quedarnos con aquellos con los que coincidan ambos años de nacimiento.


Se usa el FILTER para borrar la propia referencia a Dua Lipa de los resultados.

### Consulta 11

¿Cuántos asertos hay sobre Dua Lipa en Wikidata? Ten en cuenta que puede aparece tanto como sujeto como objeto de cada tripleta.

In [13]:
%%wdsparql
# La consulta cuenta el total de declaraciones/tripletas en las que aparece la entidad Dua Lipa (como sujeto o como objeto en la relación).
SELECT (COUNT(*) AS ?numAsertos)
WHERE {
  # Dua Lipa como sujeto       Dua Lipa como objeto.
  {wd:Q21914464 ?p ?o.} UNION {?s ?p wd:Q21914464.}
}


,numAsertos
0,1274


__Fecha de las consultas: 12 de febrero de 2025__